In [14]:
#Import Ipython that is used for display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [15]:
#Open-source Python package for exploring, visualizing, and analyzing EEGs
import mne
import numpy as np

from scipy import signal

import os, os.path
from os import listdir
from os.path import join, isdir

%matplotlib inline

In [16]:
#Ubicación del dataset
###dataset_path = './chb-mit/'
#Temporal
dataset_path = '../Previa/chb-mit/'
#Ubicación de los espectrogramas
spectrograms_path = './spectrograms/'
#Ubicación de las muestras
samples_path = "./samples/"

#verifica que la carpeta de muestras exista, en caso contrario la crea
if not os.path.exists(samples_path):
    os.makedirs(samples_path)

#Crea las rutas necesarias para entrenamiento, validación y pruebas  
path_train_0 = samples_path + "train/0/"
path_train_1 = samples_path + "train/1/"
path_val_0 = samples_path + "validation/0/"
path_val_1 = samples_path + "validation/1/"
path_test_0 = samples_path + "test/0/"
path_test_1 = samples_path + "test/1/"

#Define el tamaño de la ventana que se tomará para el espectrograma (en segundos)
window_size = 60
#Establece un overlap de 10 segundos
overlap = 10
#Se define la fase preíctal como los 20 minutos previos al ataque, es decir 20 *60 segundos
preictal = 20 * 60

In [17]:
#listdir(dataset_path)

In [18]:
"""
    Obtiene los pacientes que se usarán en el modelo
"""

dirs = [f for f in listdir(dataset_path) if isdir(join(dataset_path, f))]
print(dirs)
dirs.remove('chb04')
dirs.remove('chb05')
dirs.remove('chb06')
dirs.remove('chb07')
dirs.remove('chb08')
dirs.remove('chb09')
dirs.remove('chb10')
dirs.remove('chb11')
dirs.remove('chb12')
dirs.remove('chb13')
dirs.remove('chb14')
dirs.remove('chb15')
dirs.remove('chb16')
dirs.remove('chb17')
dirs.remove('chb18')
dirs.remove('chb19')
dirs.remove('chb20')
dirs.remove('chb21')
dirs.remove('chb22')
dirs.remove('chb23')
dirs.remove('chb24')


print(len(dirs))
print(dirs)

['chb01', 'chb02', 'chb03', 'chb04', 'chb05', 'chb06', 'chb07', 'chb08', 'chb09', 'chb10', 'chb11', 'chb12', 'chb13', 'chb14', 'chb15', 'chb16', 'chb17', 'chb18', 'chb19', 'chb20', 'chb21', 'chb22', 'chb23', 'chb24']
3
['chb01', 'chb02', 'chb03']


In [19]:
"""
    Crea un espectrograma a partir de un edf
"""
def spectrogram(raw):
    
    fs = 256
    nfft = 256
    first = 0
    
    res=0
    
    for i in range(len(raw.info.ch_names)):
        if raw.info.ch_names[i] in channels:
            
            x = raw.get_data(raw.info.ch_names[i])

            f, t, Sxx = signal.spectrogram(x, nfft=nfft, fs=fs, return_onesided=True, noverlap=128)
            
            Sxx = np.delete(Sxx, np.s_[100:129], axis=1)
            Sxx = np.delete(Sxx, np.s_[57:64], axis=1)
            
            Sxx[Sxx==0]=2.5e-78

            n = (10*np.log10(Sxx))
            n = (n-n.min())/n.ptp()
                        
            if first == 0:
                res = n
                first = 1
            else:
                res = np.vstack((res, n))
    return res

In [20]:
"""
    Canales seleccionados para ser analizados
"""

#Temporal# channels = ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ']
channels = ['FP1-F7', 'F7-T7', 'T7-P7']

n_channels = len(channels)

print(n_channels)


3


In [21]:
"""
    Crea ventanas con overlap de un espectrograma
    
    nimg: número de imagen
    path: directorio donde se guardan las ventanas
"""
def window_overlap(start, end, spec, nimg, path):    
    if overlap > 0:
        j = ((end)-(start)-(window_size*2)) // (overlap*2)
    else:
        j = ((end)-(start)-(window_size*2))
    
    if window_size*2 < (end - start) and start >= 0 and end > 0:
        for i in range(j):
            np.save(path+"img"+str(nimg), spec[:,:,start+(overlap*2*i):start+window_size*2+(i*overlap*2)])
            nimg += 1

    return nimg

In [22]:
"""
    Crea una cantidad determinada de ventanas sin overlap de un espectrograma
    
    nimg: número de imagen
    path: directorio donde se guardan las ventanas
    nimgs: número de imagenes que se crean
"""


from random import seed
from random import randint
seed(1)
    
def window(start, end, spec, nimg, path, nimgs):
    n = (end - start) // nimgs
    
    if (end-start) > window_size*2:
        for i in range(nimgs):
            #s = randint(start, end-(window_size*2))
            np.save(path+"img"+str(nimg), spec[:,:,i*n:(i*n)+(window_size*2)])
            nimg += 1
    
    return nimg

In [23]:
"""
    Cuenta los canales de un edf
"""

def count_channels(raw_file):
    n_channels = 0
    for i in range(len(raw.info.ch_names)):
        if raw.info.ch_names[i] in channels:
            n_channels += 1
    return n_channels

In [24]:
"""
    Crea los espectrogramas de cada paciente y un archivo con el inicio y fin de cada convulsión
"""

##crea el directorio donde se guardan los espectrogramas
if not os.path.exists(spectrograms_path):
    os.makedirs(spectrograms_path)

##recorre todos los pacientes
for dir in dirs:
    
    #crea el directorio de cada paciente
    if not os.path.exists(spectrograms_path+dir+"/"):
        os.makedirs(spectrograms_path+dir+"/")
    
    #path del archivo de resumen
    summary_path = dataset_path+dir+"/"+dir+"-summary.txt"
    
    #abre el archivo de resumen
    summary = open(summary_path, "r")
    
    #crea el archivo de descripción
    f = open(spectrograms_path+dir+"/"+dir+"-summary.txt", 'w')

    t = 0
    spec = None
    
    line = summary.readline()
    #lee todas las lineas del resumen
    while line != "":
        if "File Name" in line:
            
            file_name = line.split(":")[1].strip()
            file_path = dataset_path+dir+"/"+file_name
            
            #abre el archivo edf
            raw = mne.io.read_raw_edf(file_path, stim_channel=None, exclude=['-','.'])
            
            n_ch = count_channels(raw)
            
            #verifica que la cantidad de canales sea correcta
            if n_ch == n_channels:
                
                n_spec = spectrogram(raw)
                    
                #crea el espectrograma y lo adjunta a los anteriores
                if (spec is None):
                    spec = n_spec
                else:
                    spec = np.append(spec, n_spec, axis=2)

                #añade el inicio y el fin de las convulsiones al summary
                start_time = summary.readline()
                end_time = summary.readline()
                n_seizures = int(summary.readline().split(":")[1].strip())
                
                if n_seizures > 0:
                    for s in range(n_seizures):
                        next_line = summary.readline().split(":")[1].strip().split(" ")[0].strip()
                        f.write(str(int(next_line)*2+t)+"\n")
                        next_line = summary.readline().split(":")[1].strip().split(" ")[0].strip()
                        f.write(str(int(next_line)*2+t)+"\n")
                        
                t = spec.shape[2]
                print("T", t)
                
            else:
                print("El archivo tiene menos de 18 canales")
                    
        line = summary.readline();
        
    try:
        f.close()
        summary.close()
        np.save(spectrograms_path+"/"+dir+"/"+dir,spec)
        spec = None
        
    except:
        print("")
        

Extracting EDF parameters from d:\Alexis\Previa\chb-mit\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\alexi\AppData\Local\Temp/ipykernel_4208/2713942803.py:37: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path, stim_channel=None, exclude=['-','.'])


N spec [[[0.71229574 0.72946777 0.7245649  ... 0.5591091  0.59394308 0.64105123]
  [0.86455828 0.80656449 0.81448705 ... 0.79582902 0.74344495 0.61878858]
  [0.81215816 0.85700171 0.85944911 ... 0.68841725 0.68220567 0.73489411]
  ...
  [0.41270919 0.45590849 0.446923   ... 0.55147957 0.44631035 0.53950725]
  [0.48822693 0.48407247 0.42914846 ... 0.38577647 0.4372307  0.46789728]
  [0.45369768 0.49217261 0.48633988 ... 0.54940901 0.58146784 0.53294833]]

 [[0.73939461 0.72288496 0.75489952 ... 0.59776271 0.59333892 0.35528344]
  [0.86973664 0.84821363 0.82046808 ... 0.73846463 0.78854694 0.76683975]
  [0.83227767 0.8527281  0.87853241 ... 0.75586357 0.75316707 0.66009974]
  ...
  [0.58395633 0.574084   0.53022408 ... 0.576647   0.59522109 0.56612835]
  [0.59546222 0.59142031 0.60214243 ... 0.56901559 0.57469396 0.52053929]
  [0.57533777 0.58125708 0.55404826 ... 0.58493004 0.58251123 0.51465582]]

 [[0.48486431 0.65797244 0.64416004 ... 0.54152485 0.67249248 0.67082724]
  [0.89197176 0

C:\Users\alexi\AppData\Local\Temp/ipykernel_4208/2713942803.py:37: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path, stim_channel=None, exclude=['-','.'])


N spec [[[0.57836278 0.57868854 0.61445226 ... 0.6859526  0.68826277 0.50264882]
  [0.86281127 0.71458183 0.66419029 ... 0.83542677 0.82183231 0.7924577 ]
  [0.85506339 0.72054516 0.72561995 ... 0.72836593 0.7899405  0.772668  ]
  ...
  [0.5333843  0.49562093 0.41057242 ... 0.1042378  0.38690822 0.41315161]
  [0.57707914 0.48909412 0.32212325 ... 0.28556563 0.26581086 0.32205137]
  [0.47392608 0.52685218 0.54785356 ... 0.40385028 0.34816626 0.32522114]]

 [[0.56027027 0.66592095 0.64569524 ... 0.56390228 0.6861235  0.55423876]
  [0.82029311 0.74650616 0.75463136 ... 0.82151764 0.85580573 0.86190839]
  [0.78317242 0.76979481 0.74288052 ... 0.72700551 0.75889799 0.83342479]
  ...
  [0.53590147 0.51701793 0.46633133 ... 0.41080344 0.44185926 0.45127087]
  [0.535359   0.29418144 0.39968995 ... 0.41070959 0.34415738 0.41644677]
  [0.46924691 0.4761177  0.47443858 ... 0.43948974 0.39141491 0.39626966]]

 [[0.4535892  0.72049875 0.72794887 ... 0.73015528 0.6972992  0.72510357]
  [0.80290885 0

...

N spec [[[0.57325596 0.78816792 0.78406094 ... 0.5831691  0.58974561 0.53551381]
  [0.75497991 0.85572638 0.90510299 ... 0.78603651 0.77033375 0.62693997]
  [0.66382981 0.84432236 0.88740565 ... 0.71063104 0.70053736 0.6068893 ]
  ...
  [0.44138894 0.50974354 0.43986639 ... 0.39559838 0.3740719  0.45024411]
  [0.50012505 0.45466897 0.53086608 ... 0.37917551 0.45404779 0.40924937]
  [0.47392662 0.39833244 0.42799045 ... 0.45870008 0.33795018 0.40291248]]

 [[0.45793329 0.64532118 0.66347368 ... 0.67982449 0.63966319 0.59082643]
  [0.79056767 0.82100551 0.79447306 ... 0.74077641 0.79316733 0.6712592 ]
  [0.7517469  0.77327665 0.74915291 ... 0.7810087  0.75247164 0.75451208]
  ...
  [0.41742154 0.33081929 0.57687307 ... 0.46141958 0.44887034 0.45367835]
  [0.46444994 0.53337242 0.54797937 ... 0.44427705 0.49249557 0.46357932]
  [0.40861809 0.52424325 0.54154604 ... 0.47268673 0.26571981 0.44903204]]

 [[0.93755305 0.93101505 0.94128474 ... 0.93561122 0.9396984  0.93861883]
  [0.95324061 0

C:\Users\alexi\AppData\Local\Temp/ipykernel_4208/2713942803.py:37: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path, stim_channel=None, exclude=['-','.'])


N spec [[[0.80069886 0.75976299 0.64248344 ... 0.53291633 0.5897644  0.63727049]
  [0.92200571 0.85360203 0.8341548  ... 0.69305873 0.72717018 0.7285928 ]
  [0.90913616 0.80097915 0.81615407 ... 0.73732706 0.7466673  0.74702082]
  ...
  [0.50223255 0.55483976 0.52994249 ... 0.3791361  0.44514519 0.3997366 ]
  [0.50851283 0.40706507 0.52397326 ... 0.40430349 0.44480296 0.49142675]
  [0.49440914 0.50842798 0.4784277  ... 0.47879423 0.45036562 0.46283288]]

 [[0.80599401 0.78210718 0.70742735 ... 0.68366744 0.5320619  0.42277493]
  [0.92006007 0.85673196 0.83258469 ... 0.76108372 0.82799907 0.74972362]
  [0.90297376 0.82555462 0.80474995 ... 0.76738258 0.74283285 0.7435593 ]
  ...
  [0.58769482 0.56603587 0.53515931 ... 0.48605599 0.49449702 0.5187791 ]
  [0.59117531 0.43916328 0.57719322 ... 0.55132963 0.54614735 0.49887826]
  [0.61332658 0.57708355 0.58341026 ... 0.54302247 0.54203692 0.55378185]]

 [[0.70739825 0.69385065 0.4902933  ... 0.57960106 0.57311974 0.39588505]
  [0.82396654 0

C:\Users\alexi\AppData\Local\Temp/ipykernel_4208/2713942803.py:37: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path, stim_channel=None, exclude=['-','.'])


N spec [[[0.60150808 0.61935825 0.64562726 ... 0.76683091 0.5792545  0.61947264]
  [0.68512928 0.67438456 0.64382709 ... 0.87016813 0.78306759 0.68374025]
  [0.67112636 0.65241386 0.71756897 ... 0.84029904 0.63782993 0.69889941]
  ...
  [0.46493731 0.24841071 0.45553287 ... 0.5557632  0.52812437 0.40522405]
  [0.405571   0.41195801 0.41050813 ... 0.52593661 0.49835506 0.45941766]
  [0.47991279 0.42954592 0.30531304 ... 0.51282504 0.45964101 0.42498549]]

 [[0.56255506 0.579596   0.65598603 ... 0.82273852 0.50864257 0.60025806]
  [0.8043648  0.789238   0.68890108 ... 0.83377772 0.76495497 0.73667312]
  [0.75896842 0.63184733 0.74673272 ... 0.83244927 0.64943883 0.68223347]
  ...
  [0.41501128 0.46533929 0.49302414 ... 0.60347913 0.60525728 0.58077423]
  [0.44898748 0.41289019 0.45290229 ... 0.65510148 0.62064607 0.55800938]
  [0.46410966 0.42899665 0.43478776 ... 0.529034   0.58502497 0.4904338 ]]

 [[0.63701295 0.55909927 0.5862122  ... 0.76284745 0.41715587 0.68825881]
  [0.59709503 0

In [13]:
nimg_train_0 = 0
nimg_train_1 = 0
nimg_test_0 = 0
nimg_test_1 = 0

##crea el directorio donde se guardan los espectrogramas
if not os.path.exists(path_train_0):
    os.makedirs(path_train_0)
if not os.path.exists(path_train_1):
    os.makedirs(path_train_1)
if not os.path.exists(path_val_0):
    os.makedirs(path_val_0)
if not os.path.exists(path_val_1):
    os.makedirs(path_val_1)
if not os.path.exists(path_test_0):
    os.makedirs(path_test_0)
if not os.path.exists(path_test_1):
    os.makedirs(path_test_1)

##recorre todos los pacientes
for dir in dirs:
    print("Creando ventanas del paciente", dir)
    
    #path del archivo de resumen
    summary_path = spectrograms_path+dir+"/"+dir+"-summary.txt"
    
    #carga el espectrograma
    spec = np.load(spectrograms_path+dir+"/"+dir+".npy")
    
    #cantidad de convulsiones
    with open(summary_path,"r") as f:
        n_seizure = len(f.readlines()) // 2
        f.close()
    
    #abre el resumen
    summary = open(summary_path,"r")
    line = summary.readline()
    
    start = 0
    end = int(line)
    
    n_train = 0
    n_test = 0
    
    for i in range(n_seizure - 1):
        if end-start > preictal * 2:
            if overlap > 0:
                n = (((preictal * 2) - (window_size*2)) // (overlap*2)) + n_train
                n_train = 0
            else:
                n = ((preictal * 2) - (window_size*2)) + n_train
                n_train = 0
                
            nimg_train_0 = window(start, end - (preictal * 2), spec, nimg_train_0, path_train_0, n)
            nimg_train_1 = window_overlap(end - (preictal * 2), end, spec, nimg_train_1, path_train_1)
        else:
            nimg_train_1 = window_overlap(start, end, spec, nimg_train_1, path_train_1)
            if overlap > 0:
                n_train = (end-start-(window_size*2)) // (overlap*2)
            else:
                n_train = (end-start-(window_size*2))
                        
        line = summary.readline()
        start = int(line)
        
        line = summary.readline()
        end = int(line)

    if end-start > preictal * 2:
        if overlap > 0:
            n = (((preictal * 2) - (window_size*2)) // (overlap*2)) + n_test
            n_test = 0
        else:
            n = ((preictal * 2) - (window_size*2)) + n_test
            n_test = 0
            
        nimg_test_0 = window(start, end - (preictal * 2), spec, nimg_test_0, path_test_0, n)
        nimg_test_1 = window_overlap(end - (preictal * 2), end, spec, nimg_test_1, path_test_1)

    else:
        nimg_test_1a = nimg_test_1
        nimg_test_1 = window_overlap(start, end, spec, nimg_test_1, path_test_1)
        
        n = nimg_test_1 - nimg_test_1a
        line = summary.readline()
        end = int(line)
        nimg_test_0 = window(end, spec.shape[2], spec, nimg_test_0, path_test_0, n)
        """
        if overlap > 0:
            n_test = (end-start-(window_size*2)) // (overlap*2)
        else:
            n_test = (end-start-(window_size*2))
        """          
    summary.close()

Creando ventanas del paciente chb01
Creando ventanas del paciente chb02
Creando ventanas del paciente chb03
